Не решил проблему с подключением к базе, порт 5432, хост 127.0.0.1
Подскажите пожалуйста в чем еще может быть причина?

OperationalError                          Traceback (most recent call last)
<timed eval> in <module>

<ipython-input-10-1047b2b2867b> in send_sql_query(query, args)
      6     :param args: аргументы для подключения в БД.
      7     """
----> 8     conn = psycopg2.connect(**args)
      9     try:
     10         cursor = conn.cursor()

/usr/local/lib/python3.8/dist-packages/psycopg2/__init__.py in connect(dsn, connection_factory, cursor_factory, **kwargs)
    120 
    121     dsn = _ext.make_dsn(dsn, **kwargs)
--> 122     conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
    123     if cursor_factory is not None:
    124         conn.cursor_factory = cursor_factory

OperationalError: could not connect to server: Connection refused
	Is the server running on host "127.0.0.1" and accepting
	TCP/IP connections on port 5432?

In [7]:
import pandas as pd
import psycopg2
import os

In [19]:
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
# DATA_FULL_PATH = "C:\\Temp\\data\\"
DATA_FULL_PATH = "C:/Program Files/PostgreSQL/14/data/"
DATABASE_NAME = 'home_credit'

In [20]:
DB_ARGS = {
    'database': DATABASE_NAME,
    'host': '127.0.0.1',
    'user': DB_USER,
    'password': DB_PASSWORD
}

In [21]:
def send_sql_query(query: str, args: dict):
    """
    Выполняет запрос к базе.

    :param query: строка с sql запросом.
    :param args: аргументы для подключения в БД.
    """
    conn = psycopg2.connect(**args)
    try:      
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        if conn:
            cursor.close()
            conn.close()

            
def get_df_from_query(query: str, args: dict) -> pd.DataFrame:
    """
    Выполняет запрос к базе.

    :param query: строка с sql запросом.
    :param args: аргументы для подключения в БД.
    
    :return df: датафрейм с результатом.
    """  
    conn = psycopg2.connect(**args)
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [22]:
sql_schema_query = """
    DROP TABLE IF EXISTS bureau; 
    CREATE TABLE bureau(
      SK_ID_CURR integer,
      SK_ID_BUREAU integer,
      CREDIT_ACTIVE varchar(13),
      CREDIT_CURRENCY varchar(15),
      DAYS_CREDIT smallint,
      CREDIT_DAY_OVERDUE smallint,
      DAYS_CREDIT_ENDDATE real,
      DAYS_ENDDATE_FACT real,
      AMT_CREDIT_MAX_OVERDUE real,
      CNT_CREDIT_PROLONG smallint,
      AMT_CREDIT_SUM real,
      AMT_CREDIT_SUM_DEBT real,
      AMT_CREDIT_SUM_LIMIT real,
      AMT_CREDIT_SUM_OVERDUE real,
      CREDIT_TYPE varchar(49),
      DAYS_CREDIT_UPDATE integer,
      AMT_ANNUITY  real)
"""

In [23]:
%%time
send_sql_query(sql_schema_query, DB_ARGS)

OperationalError: ignored

In [13]:
sql_data_query = f"""
COPY bureau(
  SK_ID_CURR,
  SK_ID_BUREAU,
  CREDIT_ACTIVE,
  CREDIT_CURRENCY,
  DAYS_CREDIT,
  CREDIT_DAY_OVERDUE,
  DAYS_CREDIT_ENDDATE,
  DAYS_ENDDATE_FACT,
  AMT_CREDIT_MAX_OVERDUE,
  CNT_CREDIT_PROLONG,
  AMT_CREDIT_SUM,
  AMT_CREDIT_SUM_DEBT,
  AMT_CREDIT_SUM_LIMIT,
  AMT_CREDIT_SUM_OVERDUE,
  CREDIT_TYPE,
  DAYS_CREDIT_UPDATE,
  AMT_ANNUITY
)
FROM '{DATA_FULL_PATH}bureau.csv' DELIMITER ',' CSV HEADER;
"""

In [ ]:
%%time
send_sql_query(sql_data_query, DB_ARGS)

In [15]:
sql_query = """
    SELECT * FROM bureau 
"""

In [ ]:
%%time
get_df_from_query(sql_query, DB_ARGS)

In [ ]:
# вывести средний доход среди всех клиентов

%%time

query = """
SELECT AVG(amt_income_total) AS mean_income
FROM application_test;
"""
get_df_from_query(query, DB_ARGS)


In [ ]:
# вывести минимальный и максимальный возраст среди всех клиентов

%%time

query = """
SELECT MIN(YEAR(CURRENT_DATE)-YEAR(days_birth))-(RIGHT(CURRENT_DATE,5)<RIGHT(days_birth,5)) AS age,
       MAX(YEAR(CURRENT_DATE)-YEAR(days_birth))-(RIGHT(CURRENT_DATE,5)<RIGHT(days_birth,5)) AS age
FROM application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# вывести общую сумму, количество и среднюю сумму, запрошенную клиентами в кредит с авто и без

%%time

query = """
SELECT SUM(amt_credit),
       COUNT(amt_credit),
       AVG(amt_credit) FROM application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# вывести доли клиентов с различным образованием

%%time

query = """
SELECT (1 - (SELECT count(DISTINCT name_education_type = 'Higher education')
             FROM name_education_type) / count(*)) * 100
             FROM application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# подсчитать количество полных лет для клиентов, у которых есть во владении автомобиль и недвижимость.
# вывести топ 10 по возрастанию

%%time

query = """
SELECT 
    sk_id_curr,
    COUNT(*) 
    SUM(CASE WHEN name = 'flag_own_car' THEN Y ELSE N END),
    SUM(CASE WHEN name = 'flag_own_realty' THEN Y ELSE N END)
FROM 
    application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# вывести тех клиентов, у кого доход на одного члена семьи в два раза больше, чем в среднем на одного члена семьи по выборке

%%time

query = """
SELECT *
    WHERE amt_income_total / 2 > AVG(amt_income_total)
FROM 
    application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# вывести клиентов старше 60 лет по которым нет данных в bureau

%%time

query = """
SELECT *
    WHERE days_birth <= DATEADD(YEAR, -60, SYSDATETIME())
FROM 
    application_test;
"""
get_df_from_query(query, DB_ARGS)

In [ ]:
# вывести женщин, у которых в истории bureau было больше двух кредитов, просроченных на 61 день и более
# отсортировать в порядке убывания по кол-ву таких кредитов



In [ ]:
# по данным из bureau (БКИ) расчитать долю просрочки в активных займах для каждого клиента
# вывести топ 7 мужчин с наибольшей суммой просрочки, указав для них, помимо прочего,
# сумму активных кредитов и суммы всех кредитов (активных и закрытых).



In [ ]:
# 5 своих признаков для будущего обучения модели
# процент суммы кредита относительно дохода клиента
application_test['credit_income_percent'] = app_train_domain['amt_credit'] / app_train_domain['amt_income_total']

# процент аннуитетного платежа кредита относительно дохода клиента
application_test['annuity_income_percent'] = app_train_domain['amt_annuity'] / app_train_domain['amt_income_total']

# продолжительность платежа в месяцах
application_test['credit_term'] = app_train_domain['amt_annuity'] / app_train_domain['amt_credit']

# процент отработанных дней по отношению к возрасту клиента
application_test['days_employed_percent'] = app_train_domain['days_employed'] / app_train_domain['days_birth']

# полиномиальный признак давший наивысшую корреляцию с таргетом
application_test['ext_source'] = app_train_domain['ext_source_2'] * app_train_domain['ext_source_2']